In [1]:
import sys
!{sys.executable} -m pip install xarray zarr itk dask[array] toolz

In [2]:
import itk
import xarray as xr
import numpy as np
from numcodecs import Blosc
import zarr

from itkwidgets import view

In [3]:
image = itk.imread('average_template_50.nrrd')

In [4]:
view(image, vmax=300, gradient_opacity=0.1, units='μm')

Viewer(geometries=[], gradient_opacity=0.1, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy o…

In [5]:
itk.size(image)

itkSize3 ([264, 160, 228])

In [6]:
def xarray_from_image(image):
    array_view = itk.array_view_from_image(image)
    spacing = itk.spacing(image)
    origin = itk.origin(image)
    size = itk.size(image)
    direction = np.transpose(itk.array_from_matrix(image.GetDirection()))
    spatial_dimension = image.GetImageDimension()

    spatial_dims = ('x', 'y', 'z')
    coords = {}
    for index, dim in enumerate(spatial_dims[:spatial_dimension]):
        coords[dim] = np.arange(origin[index],
                                origin[index] + size[index]*spacing[index],
                                spacing[index],
                                dtype=np.float64)

    dims = list(reversed(spatial_dims[:spatial_dimension]))
    components = image.GetNumberOfComponentsPerPixel()
    if components > 1:
        dims.append('c')
        coords['c'] = np.arange(components, dtype=np.uint64)

    data_array = xr.DataArray(array_view,
                              dims=dims,
                              coords=coords,
                              attrs={'direction': direction})
    return data_array


In [7]:
image_da = xarray_from_image(image)
image_da

<xarray.DataArray (z: 228, y: 160, x: 264)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 2, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 2, 2, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]]], dtype=uint16)
Coordinates:
  * x        (x) float64 0.0 50.0 100.0 150.0 ... 1.305e+04 1.31e+04 1.315e+04
  * y        (y) float64 0.0 50.0 100.0 150.0 ... 7.85e+03 7.9e+03 7.95e+03
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 1.125e+04 1.13e+04 1.135e+04
Attributes:
    direction:  [[1. 0. 0.]\n [0. 1. 0.]\n [0. 0. 1.]]

In [8]:
image_da.chunks

In [9]:
image_ds = image_da.to_dataset(name='average_template_50')
image_ds

<xarray.Dataset>
Dimensions:              (x: 264, y: 160, z: 228)
Coordinates:
  * x                    (x) float64 0.0 50.0 100.0 ... 1.31e+04 1.315e+04
  * y                    (y) float64 0.0 50.0 100.0 ... 7.9e+03 7.95e+03
  * z                    (z) float64 0.0 50.0 100.0 ... 1.13e+04 1.135e+04
Data variables:
    average_template_50  (z, y, x) uint16 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1

In [10]:
image_ds.to_zarr(store='average_template_50.zarr', mode='w', compute=True, consolidated=True)

In [11]:
compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.SHUFFLE)
image_ds.to_zarr('average_template_50_zstd.zarr',
                  mode='w',
                  compute=True,
                  encoding = {'average_template_50': {'compressor': compressor}},
                  consolidated=True)

In [12]:
image_ds.to_zarr('average_template_50_chunked.zarr',
                  mode='w',
                  compute=True,
                  encoding = {'average_template_50': {'chunks': [64]*3, 'compressor': compressor}},
                  consolidated=True)